In [1]:
import pandas as pd
from pymongo import MongoClient
import json
from bson import json_util
from ast import literal_eval



In [2]:
viajes = pd.read_csv('viajes.csv')

# DATA

In [3]:
viajes = viajes.loc[viajes['gasto_observable']=='x']
viajes

,Unnamed: 0,codigo_viaje,tipo_de_gasto,gasto_observable,monto,inicio,fin,Duración del viaje,destino_mpio,destino_edo,coordenadas,dirección,unidad
0,0,V00001,Viáticos f,x,190.000000,25/01/2023,26/01/2023,2,León,GTO,"[-101.68121906890252, 21.119157885158575]",DG1,Dirección general
1,1,V00002,Viáticos f,x,171.000000,26/01/2023,27/01/2023,2,Morelia,MCH,"[-101.18565032966424, 19.702279138805565]",DG1,Dirección general
2,2,V00003,Viáticos f,x,232.000000,16/02/2023,17/02/2023,2,Culiacán,SIN,"[-107.3849856452668, 24.79954792100137]",DG1,Dirección general
3,3,V00004,Viáticos f,x,2042.790039,01/03/2023,02/03/2023,2,Chihuahua,CHI,"[-106.08942255973484, 28.635527801432747]",DG1,Dirección general
4,4,V00005,Viáticos f,x,116.000000,05/05/2023,05/05/2023,1,Reynosa,TMS,"[-98.27808814130742, 26.092577740486075]",DG1,Dirección general
...,...,...,...,...,...,...,...,...,...,...,...,...,...
57313,57313,V13821,Gasolina rp,x,809.640015,18/08/2023,18/08/2023,1,Axochiapan,MOR,"[-98.7530814362989, 18.50288877911706]",DG1,Estrategia
57314,57314,V12741,Viáticos rp,x,17370.000000,24/09/2023,07/10/2023,14,Morelia,MCH,"[-101.18565032966424, 19.702279138805565]",DG1,Estrategia
57315,57315,V13855,Viáticos rp,x,53.790001,01/12/2023,01/12/2023,1,Tepalcingo,MOR,"[-98.84391985481406, 18.597053735082067]",DG1,Estrategia
57316,57316,V13856,Viáticos rp,x,311.200012,08/12/2023,08/12/2023,1,Tepalcingo,MOR,"[-98.84391985481406, 18.597053735082067]",DG1,Estrategia


In [4]:
gasto_unidad=pd.DataFrame(viajes.groupby(['unidad'])['monto'].sum()).reset_index()
gasto_unidad['tipo_de_gasto']='Total'
gasto_unidad=gasto_unidad[['unidad','tipo_de_gasto','monto']]
gasto_unidad

,unidad,tipo_de_gasto,monto
0,Administración,Total,3.408728e+06
1,Atención corporativa,Total,7.005693e+05
2,Auditoría,Total,2.920409e+04
3,Auditoría interna,Total,4.640900e+05
4,Control,Total,1.086075e+05
5,Crédito,Total,3.541433e+05
6,Desarrollo y gestión,Total,3.482957e+04
7,Dirección administración,Total,1.617185e+04
8,Dirección de crédito,Total,1.078502e+05
9,Dirección de finanzas,Total,4.350357e+04


In [5]:
data1=pd.DataFrame(viajes.groupby(['unidad','tipo_de_gasto'])['monto'].sum()).reset_index()
data1

,unidad,tipo_de_gasto,monto
0,Administración,Gasolina f,32093.099883
1,Administración,Gasolina rp,363670.169649
2,Administración,Pasajes aéreos f,164693.570068
3,Administración,Pasajes aéreos rp,820410.090575
4,Administración,Pasajes terrestres f,48219.400052
...,...,...,...
229,Tesorería,Gasolina rp,3862.840058
230,Tesorería,Pasajes terrestres f,12033.579956
231,Tesorería,Pasajes terrestres rp,9337.380066
232,Tesorería,Viáticos f,51678.750517


In [6]:
data=pd.concat([data1,gasto_unidad]).reset_index()
data

,index,unidad,tipo_de_gasto,monto
0,0,Administración,Gasolina f,3.209310e+04
1,1,Administración,Gasolina rp,3.636702e+05
2,2,Administración,Pasajes aéreos f,1.646936e+05
3,3,Administración,Pasajes aéreos rp,8.204101e+05
4,4,Administración,Pasajes terrestres f,4.821940e+04
...,...,...,...,...
265,31,Riesgos,Total,5.343449e+04
266,32,Sistemas,Total,4.440407e+05
267,33,Sur,Total,6.892767e+06
268,34,Sureste,Total,3.870816e+06


In [7]:
direcciones = viajes[['unidad','dirección']]

In [8]:
direcciones['direccion_unidad']=viajes['unidad']+viajes['dirección']


C:\Users\Daniela\AppData\Local\Temp\ipykernel_25508\1823881552.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  direcciones['direccion_unidad']=viajes['unidad']+viajes['dirección']


In [9]:
direcciones_unique =direcciones.drop_duplicates(subset='direccion_unidad')
direcciones_unique.head()

,unidad,dirección,direccion_unidad
0,Dirección general,DG1,Dirección generalDG1
13,Programas y proyectos,DG5,Programas y proyectosDG5
14,Sistemas,DG6,SistemasDG6
15,Operaciones,DG6,OperacionesDG6
16,Administración,DG7,AdministraciónDG7


In [10]:
data = pd.merge(data,direcciones_unique[['unidad','dirección']],on='unidad',how='left')
data

,index,unidad,tipo_de_gasto,monto,dirección
0,0,Administración,Gasolina f,3.209310e+04,DG7
1,1,Administración,Gasolina rp,3.636702e+05,DG7
2,2,Administración,Pasajes aéreos f,1.646936e+05,DG7
3,3,Administración,Pasajes aéreos rp,8.204101e+05,DG7
4,4,Administración,Pasajes terrestres f,4.821940e+04,DG7
...,...,...,...,...,...
265,31,Riesgos,Total,5.343449e+04,DG1
266,32,Sistemas,Total,4.440407e+05,DG6
267,33,Sur,Total,6.892767e+06,DG9
268,34,Sureste,Total,3.870816e+06,DG9


In [11]:
data = data[['dirección', 'unidad', 'tipo_de_gasto', 'monto']]
data

,dirección,unidad,tipo_de_gasto,monto
0,DG7,Administración,Gasolina f,3.209310e+04
1,DG7,Administración,Gasolina rp,3.636702e+05
2,DG7,Administración,Pasajes aéreos f,1.646936e+05
3,DG7,Administración,Pasajes aéreos rp,8.204101e+05
4,DG7,Administración,Pasajes terrestres f,4.821940e+04
...,...,...,...,...
265,DG1,Riesgos,Total,5.343449e+04
266,DG6,Sistemas,Total,4.440407e+05
267,DG9,Sur,Total,6.892767e+06
268,DG9,Sureste,Total,3.870816e+06


In [12]:
data_sin_total =data.loc[data['tipo_de_gasto']!='Total']
totales=pd.DataFrame(data_sin_total.groupby(['tipo_de_gasto'])['monto'].sum()).reset_index()
totales['dirección']='Total'
totales['unidad']='Total'
totales.columns
totales=totales[['dirección', 'unidad','tipo_de_gasto', 'monto']]
totales

,dirección,unidad,tipo_de_gasto,monto
0,Total,Total,Gasolina f,5.243245e+05
1,Total,Total,Gasolina rp,7.281891e+06
2,Total,Total,Pasajes aéreos f,1.428837e+06
3,Total,Total,Pasajes aéreos rp,4.389166e+06
4,Total,Total,Pasajes terrestres f,4.482721e+05
5,Total,Total,Pasajes terrestres rp,2.975721e+06
6,Total,Total,Viáticos f,2.722788e+06
7,Total,Total,Viáticos rp,1.753991e+07


In [13]:
data=pd.concat([totales,data])
data

,dirección,unidad,tipo_de_gasto,monto
0,Total,Total,Gasolina f,5.243245e+05
1,Total,Total,Gasolina rp,7.281891e+06
2,Total,Total,Pasajes aéreos f,1.428837e+06
3,Total,Total,Pasajes aéreos rp,4.389166e+06
4,Total,Total,Pasajes terrestres f,4.482721e+05
...,...,...,...,...
265,DG1,Riesgos,Total,5.343449e+04
266,DG6,Sistemas,Total,4.440407e+05
267,DG9,Sur,Total,6.892767e+06
268,DG9,Sureste,Total,3.870816e+06


In [14]:
total_final = data.loc[data['dirección']=='Total']['monto'].sum()
total_row = {
    'dirección': ['Total'],
    'unidad': ['Total'],
    'tipo_de_gasto': ['Total'],
    'monto': [total_final]
}
total_row =pd.DataFrame(total_row)
total_row

,dirección,unidad,tipo_de_gasto,monto
0,Total,Total,Total,3.731091e+07


In [15]:
data =pd.concat([data,total_row])
data

,dirección,unidad,tipo_de_gasto,monto
0,Total,Total,Gasolina f,5.243245e+05
1,Total,Total,Gasolina rp,7.281891e+06
2,Total,Total,Pasajes aéreos f,1.428837e+06
3,Total,Total,Pasajes aéreos rp,4.389166e+06
4,Total,Total,Pasajes terrestres f,4.482721e+05
...,...,...,...,...
266,DG6,Sistemas,Total,4.440407e+05
267,DG9,Sur,Total,6.892767e+06
268,DG9,Sureste,Total,3.870816e+06
269,DG4,Tesorería,Total,1.148500e+05


# TIME DATA

In [16]:
viajes['mes']=viajes['inicio'].str[3:5]
viajes['mes']=viajes['mes'].astype(int)
viajes['mes'].unique()

array([ 1,  2,  3,  5,  6,  7,  9, 10, 11, 12,  4,  8])

In [17]:
time_data = viajes.groupby(['unidad','tipo_de_gasto','mes'])['monto'].sum().reset_index()
time_data

,unidad,tipo_de_gasto,mes,monto
0,Administración,Gasolina f,1,1891.459961
1,Administración,Gasolina f,4,1219.459961
2,Administración,Gasolina f,7,3042.880005
3,Administración,Gasolina f,8,2141.659912
4,Administración,Gasolina f,9,5776.750122
...,...,...,...,...
1235,Tesorería,Viáticos f,12,6843.500000
1236,Tesorería,Viáticos rp,1,106.000000
1237,Tesorería,Viáticos rp,4,2312.229980
1238,Tesorería,Viáticos rp,9,4518.000000


In [18]:
all_combinations = pd.DataFrame(
    [(u, t, m) for u in time_data['unidad'].unique() for t in time_data['tipo_de_gasto'].unique() for m in range(1, 13)],
    columns=['unidad', 'tipo_de_gasto', 'mes']
)

time_data = pd.merge(all_combinations, time_data, on=['unidad', 'tipo_de_gasto', 'mes'], how='left')

time_data['monto'] = time_data['monto'].fillna(0)
time_data =time_data.sort_values(by=['unidad','tipo_de_gasto','mes'])
time_data

,unidad,tipo_de_gasto,mes,monto
0,Administración,Gasolina f,1,1891.459961
1,Administración,Gasolina f,2,0.000000
2,Administración,Gasolina f,3,0.000000
3,Administración,Gasolina f,4,1219.459961
4,Administración,Gasolina f,5,0.000000
...,...,...,...,...
3451,Tesorería,Viáticos rp,8,0.000000
3452,Tesorería,Viáticos rp,9,4518.000000
3453,Tesorería,Viáticos rp,10,0.000000
3454,Tesorería,Viáticos rp,11,19816.109711


In [27]:
time_data_total = time_data.groupby(['tipo_de_gasto','mes'])['monto'].sum().reset_index()
time_data_total['unidad']='Total'
time_data_total = time_data_total[['unidad','tipo_de_gasto','mes','monto']]
time_data_total

,unidad,tipo_de_gasto,mes,monto
0,Total,Gasolina f,1,2.125189e+04
1,Total,Gasolina f,2,3.568469e+04
2,Total,Gasolina f,3,3.132226e+04
3,Total,Gasolina f,4,6.908940e+03
4,Total,Gasolina f,5,1.098935e+04
...,...,...,...,...
91,Total,Viáticos rp,8,1.501064e+06
92,Total,Viáticos rp,9,3.147632e+06
93,Total,Viáticos rp,10,3.278711e+06
94,Total,Viáticos rp,11,4.024916e+06


In [28]:
time_data =pd.concat([time_data_total,time_data])
time_data

,unidad,tipo_de_gasto,mes,monto
0,Total,Gasolina f,1,21251.889954
1,Total,Gasolina f,2,35684.689970
2,Total,Gasolina f,3,31322.260011
3,Total,Gasolina f,4,6908.939880
4,Total,Gasolina f,5,10989.349914
...,...,...,...,...
3451,Tesorería,Viáticos rp,8,0.000000
3452,Tesorería,Viáticos rp,9,4518.000000
3453,Tesorería,Viáticos rp,10,0.000000
3454,Tesorería,Viáticos rp,11,19816.109711


In [30]:
time_data['monto_acumulado'] = time_data.groupby(['unidad','tipo_de_gasto'])['monto'].cumsum()

In [31]:
time_data.loc[time_data['unidad']=='Total']

,unidad,tipo_de_gasto,mes,monto,monto_acumulado
0,Total,Gasolina f,1,2.125189e+04,2.125189e+04
1,Total,Gasolina f,2,3.568469e+04,5.693658e+04
2,Total,Gasolina f,3,3.132226e+04,8.825884e+04
3,Total,Gasolina f,4,6.908940e+03,9.516778e+04
4,Total,Gasolina f,5,1.098935e+04,1.061571e+05
...,...,...,...,...,...
91,Total,Viáticos rp,8,1.501064e+06,4.691642e+06
92,Total,Viáticos rp,9,3.147632e+06,7.839274e+06
93,Total,Viáticos rp,10,3.278711e+06,1.111799e+07
94,Total,Viáticos rp,11,4.024916e+06,1.514290e+07


# LOCATION_DATA

In [24]:
duplicates_mask = viajes.duplicated('codigo_viaje', keep='first')
location_data = viajes[~duplicates_mask]
location_data = location_data[['codigo_viaje','inicio', 'fin', 'Duración del viaje', 'destino_mpio',
       'destino_edo', 'coordenadas', 'dirección', 'unidad', 'mes']]
location_data['coordenadas'] = location_data['coordenadas'].apply(lambda x: literal_eval(x) if pd.notna(x) else x)

location_data

,codigo_viaje,inicio,fin,Duración del viaje,destino_mpio,destino_edo,coordenadas,dirección,unidad,mes
0,V00001,25/01/2023,26/01/2023,2,León,GTO,"[-101.68121906890252, 21.119157885158575]",DG1,Dirección general,1
1,V00002,26/01/2023,27/01/2023,2,Morelia,MCH,"[-101.18565032966424, 19.702279138805565]",DG1,Dirección general,1
2,V00003,16/02/2023,17/02/2023,2,Culiacán,SIN,"[-107.3849856452668, 24.79954792100137]",DG1,Dirección general,2
3,V00004,01/03/2023,02/03/2023,2,Chihuahua,CHI,"[-106.08942255973484, 28.635527801432747]",DG1,Dirección general,3
4,V00005,05/05/2023,05/05/2023,1,Reynosa,TMS,"[-98.27808814130742, 26.092577740486075]",DG1,Dirección general,5
...,...,...,...,...,...,...,...,...,...,...
57208,V16990,16/06/2023,16/06/2023,1,Hueytamalco,PUE,"[-97.28721141722377, 19.93754525357293]",DG9,Sur,6
57217,V16991,08/11/2023,08/11/2023,1,Irapuato,GTO,"[-101.34815568631001, 20.673625916155615]",DG1,Estrategia,11
57278,V16992,15/09/2023,15/09/2023,1,Tuxpan,NAY,"[-105.29965490950036, 21.943832942215654]",DG9,Occidente,9
57302,V16993,16/11/2023,16/11/2023,1,Loreto,BCS,"[-111.34398674035934, 26.0130952521972]",DG9,Noroeste,11


In [31]:
location_data['destino_mpio']=location_data['destino_mpio'].str.upper()
location_data['mpio_edo'] = location_data['destino_mpio']+'-'+location_data['destino_edo']
location_data=location_data.rename(columns={
    'Duración del viaje':'duracion'
})
location_data

,codigo_viaje,inicio,fin,duracion,destino_mpio,destino_edo,coordenadas,dirección,unidad,mes,mpio_edo
0,V00001,25/01/2023,26/01/2023,2,LEÓN,GTO,"[-101.68121906890252, 21.119157885158575]",DG1,Dirección general,1,LEÓN-GTO
1,V00002,26/01/2023,27/01/2023,2,MORELIA,MCH,"[-101.18565032966424, 19.702279138805565]",DG1,Dirección general,1,MORELIA-MCH
2,V00003,16/02/2023,17/02/2023,2,CULIACÁN,SIN,"[-107.3849856452668, 24.79954792100137]",DG1,Dirección general,2,CULIACÁN-SIN
3,V00004,01/03/2023,02/03/2023,2,CHIHUAHUA,CHI,"[-106.08942255973484, 28.635527801432747]",DG1,Dirección general,3,CHIHUAHUA-CHI
4,V00005,05/05/2023,05/05/2023,1,REYNOSA,TMS,"[-98.27808814130742, 26.092577740486075]",DG1,Dirección general,5,REYNOSA-TMS
...,...,...,...,...,...,...,...,...,...,...,...
57208,V16990,16/06/2023,16/06/2023,1,HUEYTAMALCO,PUE,"[-97.28721141722377, 19.93754525357293]",DG9,Sur,6,HUEYTAMALCO-PUE
57217,V16991,08/11/2023,08/11/2023,1,IRAPUATO,GTO,"[-101.34815568631001, 20.673625916155615]",DG1,Estrategia,11,IRAPUATO-GTO
57278,V16992,15/09/2023,15/09/2023,1,TUXPAN,NAY,"[-105.29965490950036, 21.943832942215654]",DG9,Occidente,9,TUXPAN-NAY
57302,V16993,16/11/2023,16/11/2023,1,LORETO,BCS,"[-111.34398674035934, 26.0130952521972]",DG9,Noroeste,11,LORETO-BCS


# SEND DATAFRAMES TO MONGO DATABASE

In [47]:
data_dict =data.to_dict(orient='records')
client =MongoClient('mongodb://localhost:27017/')
db = client['viajes']
collection = db['data']
collection.insert_many(data_dict)
client.close()

In [32]:
time_data_dict =time_data.to_dict(orient='records')
client =MongoClient('mongodb://localhost:27017/')
db = client['viajes']
collection = db['time_data']
collection.insert_many(time_data_dict)
client.close()

In [71]:
location_data_dict =location_data.to_dict(orient='records')
client =MongoClient('mongodb://localhost:27017/')
db = client['viajes']
collection = db['location_data']
collection.insert_many(location_data_dict)
client.close()